# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
all_databases = mongo.list_database_names()


if 'uk_food' in all_databases:
    print("'uk_food' database exists!")


'uk_food' database exists!


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
collections = db.list_collection_names()
print(collections)

['establishments']


In [6]:
# review the collections in our new database
establishment = db.establishments.find_one()
pprint(establishment)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64ecd86494aee36a68d145b7'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

In [7]:
# review a document in the establishments collection
establishment = db.establishments.find_one()
pprint(establishment)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64ecd86494aee36a68d145b7'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "AddressLine3": "Greenwich",
    "BusinessType": "halal"
}

print(new_restaurant["BusinessName"])

Penang Flavours


In [10]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)

In [11]:
# Check that the new restaurant was inserted
if result.acknowledged:
    print(f"New restaurant was successfully inserted") 
else:
    print("There was an error inserting the new restaurant.")

New restaurant was successfully inserted


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}  # Projection to return only the specified fields and exclude _id
)

pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
filter_criteria = {"BusinessName": "Penang Flavours"}
update_operation = {
    "$set": {"BusinessTypeID": 1}
}

In [18]:
# Confirm that the new restaurant was updated
result = establishments.update_one(filter_criteria, update_operation)

if result.modified_count > 0:
    print("Updated record(s) successfully!")
else:
    print("No records were updated.")



No records were updated.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [22]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Number of documents:", dover_count)


Number of documents: 0


In [23]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})

print(f"Number of establishments deleted with LocalAuthorityName as 'Dover': {delete_result.deleted_count}")

Number of establishments deleted with LocalAuthorityName as 'Dover': 0


In [27]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})

if remaining_dover_count == 0:
    print("No establishments with LocalAuthorityName as 'Dover'  in the database.")
else:
    print("There is establishments with LocalAuthorityName as 'Dover'  in the database.")

No establishments with LocalAuthorityName as 'Dover'  in the database.


In [28]:
# Check that other documents remain with 'find_one'
remain_document = establishments.find_one()
remain_document

{'_id': ObjectId('64ecd86494aee36a68d145b7'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [29]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"},
                                         'geocode.latitude': {"$toDouble": '$geocode.latitude'}}}])

Use `update_many` to convert `RatingValue` to integer numbers.

In [30]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [31]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}
)


In [32]:
# Check that the coordinates and rating value are now numbers
query = {}
geo_field = {"geocode.longitude":1, "geocode.latitude":1}
pprint(establishments.find_one(query, geo_field))

{'_id': ObjectId('64ecd86494aee36a68d145b7'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}}
